In [4]:
%load_ext autoreload
%autoreload 2

!export TRITON_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/triton
!export XDG_CONFIG_HOME=/data/gpfs/projects/punim2662/.config
!export VLLM_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/vllm
!export VLLM_CACHE_ROOT=/data/gpfs/projects/punim2662/.cache/vllm
!export TORCH_HOME=/data/gpfs/projects/punim2662/.cache/torch/
!export TORCHINDUCTOR_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/torch/inductor
!export CUDA_CACHE_PATH=/data/gpfs/projects/punim2662/.cache/nvidia/
!export HF_HOME=/data/gpfs/projects/punim2662/.cache/huggingface

# Hugging Face authentication token - set via environment variable before running
# Example: export HF_AUTH_TOKEN="your_token_here" in your shell before starting Jupyter
# Or set it in this notebook: import os; os.environ['HF_AUTH_TOKEN'] = "your_token_here"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### **Load the mmlu auxiliary split**


In [5]:
from routing_dataset.dataset_paths import *
import pickle
from datasets import load_dataset, get_dataset_config_names, Dataset
from routing_dataset.load_dataset import load_mmlu_pro_dataset
from routing_dataset.load_dataset import load_gsm8k_split
from routing_dataset.run_prompts import add_correct_labels
import pandas

In [ ]:
from routing_dataset.dataset_paths import (
    FINAL_HOTPOTQA_QWEN8B_TRAIN_FILE,
    FINAL_HOTPOTQA_QWEN8B_VAL_FILE,
    FINAL_HOTPOTQA_QWEN8B_TEST_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TRAIN_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_VAL_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TEST_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TRAIN_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_VAL_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TEST_FILE,
    APPS_PROMPTS_FILE 
)
from matplotlib import pyplot as plt



from datasets import load_dataset

humaneval_ds = load_dataset("openai_humaneval")
mbpp_ds = load_dataset("google-research-datasets/mbpp")

apps = load_dataset("codeparrot/apps")



In [35]:
from routing_dataset.dataset_paths import (
    APPS_PROMPTS_WITH_CORRECT_LABELS_QWEN8B_FILE,
    APPS_QWEN8B_TRAIN_FILE,
    APPS_QWEN8B_VAL_FILE,
    APPS_QWEN8B_TEST_FILE
)
from routing_dataset.load_dataset import stratfieid_train_validation_split
from routing_dataset.run_apps_vllm import format_apps_prompt_qwen
with open(APPS_PROMPTS_WITH_CORRECT_LABELS_QWEN8B_FILE, 'rb') as f:
    apps_prompts = pickle.load(f)

apps_prompts = pandas.DataFrame(apps_prompts)
# apply format_apps_prompt_qwen to all the prompts
apps_prompts['prompts'] = apps_prompts['prompts'].apply(format_apps_prompt_qwen)

apps_prompts['dataset_source'] = 'apps'
train, test, valdation = stratfieid_train_validation_split(apps_prompts, 0.8)

# save the splits
train.to_pickle(APPS_QWEN8B_TRAIN_FILE)
test.to_pickle(APPS_QWEN8B_TEST_FILE)
valdation.to_pickle(APPS_QWEN8B_VAL_FILE)


In [36]:
with open(APPS_QWEN8B_TRAIN_FILE, 'rb') as f:
    train = pickle.load(f)

print(train.keys())

Index(['problem_ids', 'prompts', 'formatted_prompts', 'responses',
       'starter_codes', 'solutions', 'difficulty', 'url', 'input_output',
       'correct_labels', 'judge_reasons', 'dataset_source'],
      dtype='object')
